# ImageNet Classification
>Performance analysis for ImageNet Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [ImageNet,Rooflines,Performance Prediction]
- image: images/imagenet_logo.png

In [1]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480
pd.options.display.float_format = '{:20,.6f}'.format
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 30

csv_path = "./data/cleaned_csv/backup.csv"

In [2]:
#hide
# master_df.loc[(master_df.NN_Topology =='RN50') & (master_df.PruningFactor != '100%')]

# Theoretical Analysis of ImageNet

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [3]:
#hide_input

#first process the following csv's to get clean ready-to-plot csv's
%run scripts/script_load_save_data.py
clean_csv_rooflines(path_topologies='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/topology_details.csv',
                    path_hardware='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/peakPerfBandHardPlatf.csv')

#Now get the cleaned csv, and plot it as a Roofline
%run scripts/altair_plots.py
rooflines(dataframe = pd.read_csv("data/cleaned_csv/rooflines_hardware_neural_networks.csv"), 
          neural_network = 'imagenet')

alt.LayerChart(...)

### Performance Prediction

The following heatmap shows the theoretical performance for the listed hardware platforms for ImageNet classification. The metric used for the theoretical performance is input/second.
Looking at the plot, it becomes clear that prunning along with quantization outputs some of the best performance results.

In [4]:
#hide_input
path_csv = 'data/performance_predictions_imagenet_mnist_cifar_2.csv'
    ## Reading csv file and converting data to (Neural network, Platform, Value)
df = pd.read_csv(path_csv)

df1 = pd.DataFrame()
columns = (df.loc[:, df.columns!='hardw']).columns #select all columns except first
for column in columns:
    df_=pd.melt(df, id_vars=['hardw'], value_vars=column) #melt df1 into a df1 of 2 columns
    df1=pd.concat([df1,df_])
df1.columns= ['y','x','values'] #setting new column names
#replace 0s for NaN values because with 0s the grid doesn't show up
df1['values'] = df1['values'].replace({ 0.0:np.nan})
df_imagenet = dataframe_contains(input_df=df1, column='x', value='GoogleNetv|MobileNetv1|ResNet50|EfficientNet')

df_imagenet.to_csv('c:/Users/alinav/Documents/GitHub/QutibenchWeb/_notebooks/data/cleaned_csv/performance_prediction_imagenet.csv', index = False)

#df_cifar10.head(200)
heatmap(df_imagenet, 'pink', 'Performance prediction for ImageNet')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for ImageNet

In this table, within the rows, we show the type of hardware platforms that we used for this task (for example FPGA or GPU) and then more specifically the exact name of the different hardware platforms. For each hardware platform, we list the sweep of specific deployment parameters (batch sizes, operating modes etc) that were used for the experimentation in separate columns. In the columns, we show CNN topologies. When a CNN topology was implemented on a given hardware platform, we show in the corresponding cell the precisions (quantization information) and the channel pruning scale. Otherwise, “na” indicates that the topology wasn’t executed on this specific hardware platform. Many combinations between topology and hardware platform are not supported by the vendors dedicated software environments. INTx depicts a fixed point integer representation with x bits. FPy represents a floating point representation with y bits, for example FP32 is singe precision floating point. Table follows below.

In [5]:
#hide
print(pd.read_csv('data/overview_experiments_imagenet.csv').to_markdown())

|    | Hardware   | Platform         | ResNet50                  | GoogLeNetV1   | MobileNet   | Batch/Stream/Thread                  |
|---:|:-----------|:-----------------|:--------------------------|:--------------|:------------|:-------------------------------------|
|  0 | FPGA       | ZCU102-DPU       | [INT8]*[100%,80%,50%,30%] | INT8          | na          | [1,2,3,4,5,6,7,8]                    |
|  1 | FPGA       | ZCU104-DPU       | INT8                      | INT8          | na          | [1,2,3,4,5,6,7,8]                    |
|  2 | FPGA       | Ultra96-DPU      | [INT8]*[100%,80%,50%,30%] | INT8          | INT8        | [1,2,3,4,5,6,7,8]                    |
|  3 | FPGA       | ZCU104-FINN      | na                        | na            | na          | [1,2,4,8,16,32,64,128,256,512,10000] |
|  4 | FPGA       | ZCU104-BISMO     | na                        | na            | na          | [2,4,8,16,32,64,128]                 |
|  5 | GPU        | TX2-maxn         | FP16,FP32

In [6]:
#hide_input
%run scripts/script_tables.py 
#get table with the experiments overview
dataframes = csv_to_dataframe_multiindex(['data/overview_experiments_imagenet_.csv'])
for dataframe in dataframes:   
       display(HTML(dataframe.to_html(index=False)))

In [7]:
#hide
master_df = pd.read_csv(csv_path)
#fix ResNet50 Pruning values from 100,50,25,12.5 to -> 100,80,50,30
is_maxp = lambda row: row.HWType != "TX2" or row["Op mode"].split(",")[0] == "maxp" or row["Op mode"] == "fast" or row["Op mode"] == "slow"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
imagenet_df = maxp_df[maxp_df.NN_Topology.isin(['GNv1','RN50','MNv1','RN50V15']) & maxp_df['lat-comp'].notna()]
bad_precisions = ["FP"+str(i) for i in range(17,24)]
#this version has the values for ResNet50 v1.5
imagenet_df.Datatype = imagenet_df.Datatype.apply(lambda x: 'FP16' if x in bad_precisions else x)
imagenet_df["hw_datatype_prun_net"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor, r.NN_Topology]), axis=1)

imagenet_df["PruningFactor"] = imagenet_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(imagenet_df, "lat-comp", "NN_Topology");
imagenet_df["datatype_model"] = imagenet_df.Datatype + '_' + imagenet_df.HWType
imagenet_df.rename(columns={"top1 [%]": "top1"}, inplace=True)
imagenet_df["tag"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.NN_Topology, str(r.PruningFactor)]), axis=1)

#filling GOPS data gaps 
imagenet_df['GOPS'] = imagenet_df.apply(lambda r: 1.14 if r.NN_Topology == 'MNv1' else 
                                          ( 3.13 if r.NN_Topology == 'GNv1' else 
                                           ( 8.2 if r.NN_Topology == 'RN50V15' else 
                                            ( 7.72 if r.NN_Topology == 'RN50' and r.PruningFactor == 100 else 
                                             ( 6.54 if r.NN_Topology == 'RN50' and r.PruningFactor == 80 else 
                                              ( 3.75 if r.NN_Topology == 'RN50' and r.PruningFactor == 50 else 
                                               ( 2.45 if r.NN_Topology == 'RN50' and r.PruningFactor == 30 else 0 )))))) , axis=1)

#fill in tp-system and tp-cmp
imagenet_df['tp-system'] = imagenet_df['fps-system'] * imagenet_df['GOPS']
imagenet_df['tp-comp'] = imagenet_df['fps-comp'] * imagenet_df['GOPS']
imagenet_df['GOPS'] = imagenet_df['GOPS'] * imagenet_df['batch/thread/stream']
imagenet_df.head(300)

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy o

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_datatype_prun_net,norm-lat-comp,datatype_model,tag
0,RN50V15,EdgeTPU,INT8,fast,1,NaN,40.441510,10.552000,24.727000,86.526400,202.761400,nan,nan,nan,nan,1.190000,8.200000,100.000000,l3,EdgeTPU_INT8_100%_RN50V15,0.954757,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
1,RN50V15,EdgeTPU,INT8,fast,1,NaN,40.585040,10.589000,24.640000,86.829800,202.048000,nan,nan,nan,nan,1.490000,8.200000,100.000000,l3,EdgeTPU_INT8_100%_RN50V15,0.958145,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
2,RN50V15,EdgeTPU,INT8,slow,1,NaN,42.357920,10.075000,23.608000,82.615000,193.585600,nan,nan,nan,nan,0.962300,8.200000,100.000000,l3,EdgeTPU_INT8_100%_RN50V15,1.000000,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
3,RN50V15,EdgeTPU,INT8,slow,1,NaN,41.695590,7.111000,23.983000,58.310200,196.660600,nan,nan,nan,nan,1.020000,8.200000,100.000000,l3,EdgeTPU_INT8_100%_RN50V15,0.984363,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
273,MNv1,TPU,INT8,slow,1,7.86,4.082490,127.256000,244.949000,145.071840,279.241860,69.567400,87.705800,0.253000,0.253000,0.462000,1.140000,100.000000,l3,TPU_INT8_100%_MNv1,1.000000,INT8_TPU,TPU_INT8_MNv1_100.0
274,MNv1,TPU,INT8,fast,1,6,2.570470,166.533000,389.034000,189.847620,443.498760,69.567400,87.705800,0.253000,0.253000,0.532000,1.140000,100.000000,l3,TPU_INT8_100%_MNv1,0.629633,INT8_TPU,TPU_INT8_MNv1_100.0
403,GNv1,TPU,INT8,slow,1,10.03,5.721310,99.741000,174.785000,312.189330,547.077050,69.243400,88.445800,0.253000,0.253000,0.463000,3.130000,100.000000,l3,TPU_INT8_100%_GNv1,0.006099,INT8_TPU,TPU_INT8_GNv1_100.0
404,GNv1,TPU,INT8,fast,1,7.4,3.648520,135.087000,274.084000,422.822310,857.882920,69.243400,88.445800,0.253000,0.253000,0.538000,3.130000,100.000000,l3,TPU_INT8_100%_GNv1,0.003890,INT8_TPU,TPU_INT8_GNv1_100.0
437,GNv1,TX2,FP16,maxp,1,9.93,6.163370,99.824500,169.338000,312.450685,530.027940,66.928000,87.832000,1.800000,4.700000,8.070000,3.130000,100.000000,l3,TX2_FP16_100%_GNv1,0.006571,FP16_TX2,TX2_FP16_GNv1_100.0
438,GNv1,TX2,FP16,maxp,2,17.06,10.619700,108.360000,192.363000,339.166800,602.096190,66.928000,87.832000,1.800000,4.700000,8.280000,6.260000,100.000000,l3,TX2_FP16_100%_GNv1,0.011321,FP16_TX2,TX2_FP16_GNv1_100.0


### Line Plot

In [8]:
#hide_input
dataframe = imagenet_df
sel = alt.selection_multi(fields=["hw_datatype_prun_net"], bind="legend")
fig25_dot = alt.Chart(dataframe).mark_point().encode(
    x='lat-comp',
    y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
    color=select_color(sel, 'hw_datatype_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)
fig25_line = alt.Chart(dataframe).mark_line().encode(
    x='lat-comp',
    y='fps-comp',
    color=select_color(sel, 'hw_datatype_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)

fig = (fig25_dot+fig25_line).properties(
    title="Latency versus Performance for Pruned and Quantized Imagenet Classifier Variants",
    width=W,
    height=H,
).add_selection(sel).interactive()

fig

alt.LayerChart(...)

### Boxplots

In [9]:
#hide_input 
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=imagenet_df, xaxis='PruningFactor', yaxis="lat-comp", color_col= 'PruningFactor', facet_column='datatype_model' , title="Latency by Hardware/Framework and Pruning for Imagenet Classification")

alt.FacetChart(...)

In [10]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=imagenet_df, xaxis='PruningFactor', yaxis="fps-comp", color_col= 'PruningFactor', facet_column='datatype_model' , title="Throughput by Hardware/Framework and Pruning for Imagenet Classification")

alt.FacetChart(...)

In [11]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=imagenet_df, xaxis='PruningFactor', yaxis="Full_Pwr_W", color_col= 'PruningFactor', facet_column='datatype_model' , title="Power Consumption by Hardware/Framework and Pruning for Imagenet Classification")

alt.FacetChart(...)

## Pareto Graphs

The following pareto graph presents the accuracy versus performance in fps for all the Hardware Platforms across different Pruning and Quantization configurations. This provides insights into accuracy-based comparisons.

In [12]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
# ResNet50 v15 does not have accuracy measurements yet, so it needs to be taken out
df_pareto_graph = imagenet_df[imagenet_df.NN_Topology != 'RN50V15']
pareto_graph(df= df_pareto_graph, 
             groupcol= 'tag', 
             xcol= 'fps-comp', 
             ycol= 'top1', 
             W= W, 
             H= H, 
             title= "ImageNet Cassification Design Space: Accuracy versus Performance")


alt.LayerChart(...)

In [13]:
#hide
def replace_data_df(df_: pd.DataFrame(), column:str, list_tuples_data_to_replace: list )-> pd.DataFrame():
    """Method to replace a substring inside a cell inside a dataframe"""
    df = df_.copy()
    for j, k in list_tuples_data_to_replace:
        df[column] = df[column].str.replace(j, k)
    return df

In [14]:
#hide
""" This is to create a df to be joined with the theoretical df in 'Theoretical Analysis' to create the overlapped paretos

Steps
------
1. Create subset from imagenet that doesn't have the ResNet50 v15 measurements because it does not have accuracy measures
2. Create new hardware column that has hardware and operation mode, beware with NaNs
3. Create new 'hardw_datatype_net_prun' with hardware + datatype + netwrok + pruning
4. Create a suset of the dataframe with the above mentioned column and the corresponding ones
5. With groupby for col 'hardw_datatype_net_prun', for each unique value get the rows with biggest batch 
6. Add 'type column', reset the index from 'hardw_datatype_net_prun' to ints and save it
"""
imagenet_df_tmp = imagenet_df.copy()
# ResNet50 v15 does not have accuracy measurements yet, so it needs to be taken out
# create imagenet_df_tmp from imagenet_df
imagenet_df_tmp = imagenet_df_tmp[imagenet_df_tmp.NN_Topology != 'RN50V15']
# create hardw column to include: hardware + op_mode
imagenet_df_tmp['hardw'] = imagenet_df_tmp['HWType'] + ('-' + imagenet_df_tmp['Op mode']).fillna('')
#create hardw_datatype_net_prun col with all those columns merged
imagenet_df_tmp['hardw_datatype_net_prun'] = imagenet_df_tmp.apply(lambda r: "_".join([r.hardw, r.Datatype, r.NN_Topology, str(r.PruningFactor)]), axis=1)
#create a subset of the dataframe with only those columns
imagenet_df_tmp = imagenet_df_tmp[['hardw_datatype_net_prun','hardw', 'NN_Topology' ,'fps-comp', 'top1','batch/thread/stream']]
#Only get the points corresponding to the biggest batch
imagenet_df_tmp = imagenet_df_tmp.groupby('hardw_datatype_net_prun')[['batch/thread/stream','hardw', 'NN_Topology','fps-comp', 'top1']].max()
#add type column
imagenet_df_tmp['type'] = 'measured'
# reset index to start being numeric 
imagenet_df_tmp = imagenet_df_tmp.reset_index()
#save it all
imagenet_df_tmp.to_csv('data/cleaned_csv/pareto_data_imagenet.csv', index = False)
#   change column names
imagenet_df_tmp.columns = ['hardw_datatype_net_prun', 'batch/thread/stream', 'hardw', 'network', 'fps-comp', 'top1', 'type']
#   fix samll stuff in the df so things match with the other side
imagenet_df_tmp = replace_data_df(df_=imagenet_df_tmp, column='hardw_datatype_net_prun', list_tuples_data_to_replace=[("RN50", "ResNet50"),("MNv1", "MobileNetv1"),('GNv1','GoogLeNetv1'),('100.0','100'),('25.0','25') ,('50.0','50')])
imagenet_df_tmp = replace_data_df(df_=imagenet_df_tmp, column='network', list_tuples_data_to_replace=[("RN50", "ResNet50"),("MNv1", "MobileNetv1"),('GNv1','GoogLeNetv1')])
#delete unnecessary columns
imagenet_df_tmp = imagenet_df_tmp.drop(columns=['batch/thread/stream'])

#save it all
imagenet_df_tmp.to_csv('data/cleaned_csv/pareto_data_imagenet.csv', index = False)

#plot it just to be sure everything os okay
pareto_graph(df= imagenet_df_tmp, 
             groupcol= 'hardw_datatype_net_prun', 
             xcol= 'fps-comp', 
             ycol= 'top1', 
             W= W, 
             H= H, 
             title= "ImageNet Cassification Design Space: Accuracy versus Performance")

imagenet_df_tmp

,hardw_datatype_net_prun,hardw,network,fps-comp,top1,type
0,NCS_FP16_ResNet50_100,NCS,ResNet50,18.271600,75.172000,measured
1,TPU-fast_INT8_GoogLeNetv1_100,TPU-fast,GoogLeNetv1,274.084000,69.243400,measured
2,TPU-fast_INT8_MobileNetv1_100,TPU-fast,MobileNetv1,389.034000,69.567400,measured
3,TPU-slow_INT8_GoogLeNetv1_100,TPU-slow,GoogLeNetv1,174.785000,69.243400,measured
4,TPU-slow_INT8_MobileNetv1_100,TPU-slow,MobileNetv1,244.949000,69.567400,measured
5,TX2-maxp_FP16_GoogLeNetv1_100,TX2-maxp,GoogLeNetv1,240.699000,66.928000,measured
6,TX2-maxp_FP16_ResNet50_100,TX2-maxp,ResNet50,114.033000,75.142000,measured
7,TX2-maxp_FP32_GoogLeNetv1_100,TX2-maxp,GoogLeNetv1,136.319000,66.956000,measured
8,TX2-maxp_FP32_ResNet50_100,TX2-maxp,ResNet50,61.514100,75.148000,measured
9,Ultra96-DPU_INT8_GoogLeNetv1_100,Ultra96-DPU,GoogLeNetv1,66.509500,69.410000,measured


# Theoretical Pareto and Measured Pareto Overlapped

In order to easily understand how accurate predictions were, an overlapping between the Theoretical Pareto Plot and Measured Pareto Plot was made. In the plot below we have both theoretical (orange) and measured (blue) pareto lines. All measured datapoins are represented as crosses and all theoretical datatpoins are represented as circles. Some theoretical datapoints don't have a measured matched datapoint and the same goes for the measured datapoints. The theoretical pareto curve is, as expected, on the right of the measured one, as predictions are sometimes different form measurements.

In [15]:
#hide_input
%run scripts/overlapped_pareto.py
get_overlapped_pareto('imagenet')

alt.LayerChart(...)

# Efficiency Plot

In order to understand the gap between the theoretical predictions and what was measured, an efficiency bar-chart was created. The size of the bar reflects the absolute performance, whereby all theoretical predictions are shown in red, theoretical peak performance in blue, and all measured datapoints in orange. The orange bars are annotated with the efficiency achieved as a percentage of the predicted performance. Please note the logarithmic y-axis scale. The theoretical predictions take memory bottlenecks into account, as such measured performance can actually exceed the predicted result, in which case the percentage can be above 100%.

In [16]:
#hide_input
%run scripts/overlapped_pareto.py
imagenet_efficiency_df = get_peak_perf_gops_df(df_=imagenet_df_tmp) #takes the imagenet df and fills it with data for the 3rd bar - Theoretical Peak Performance
efficiency_plot(net_keyword= 'imagenet', df_theo_peak_compute=imagenet_efficiency_df, title='Efficiency plots for ImageNet')

alt.FacetChart(...)

In [17]:
#hide
imagenet_df.to_csv('data/cleaned_csv/experimental_data_imagenet.csv', index = False)
imagenet_df

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_datatype_prun_net,norm-lat-comp,datatype_model,tag
0,RN50V15,EdgeTPU,INT8,fast,1,NaN,40.441510,10.552000,24.727000,86.526400,202.761400,nan,nan,nan,nan,1.190000,8.200000,100.000000,l3,EdgeTPU_INT8_100%_RN50V15,0.954757,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
1,RN50V15,EdgeTPU,INT8,fast,1,NaN,40.585040,10.589000,24.640000,86.829800,202.048000,nan,nan,nan,nan,1.490000,8.200000,100.000000,l3,EdgeTPU_INT8_100%_RN50V15,0.958145,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
2,RN50V15,EdgeTPU,INT8,slow,1,NaN,42.357920,10.075000,23.608000,82.615000,193.585600,nan,nan,nan,nan,0.962300,8.200000,100.000000,l3,EdgeTPU_INT8_100%_RN50V15,1.000000,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
3,RN50V15,EdgeTPU,INT8,slow,1,NaN,41.695590,7.111000,23.983000,58.310200,196.660600,nan,nan,nan,nan,1.020000,8.200000,100.000000,l3,EdgeTPU_INT8_100%_RN50V15,0.984363,INT8_EdgeTPU,EdgeTPU_INT8_RN50V15_100.0
273,MNv1,TPU,INT8,slow,1,7.86,4.082490,127.256000,244.949000,145.071840,279.241860,69.567400,87.705800,0.253000,0.253000,0.462000,1.140000,100.000000,l3,TPU_INT8_100%_MNv1,1.000000,INT8_TPU,TPU_INT8_MNv1_100.0
274,MNv1,TPU,INT8,fast,1,6,2.570470,166.533000,389.034000,189.847620,443.498760,69.567400,87.705800,0.253000,0.253000,0.532000,1.140000,100.000000,l3,TPU_INT8_100%_MNv1,0.629633,INT8_TPU,TPU_INT8_MNv1_100.0
403,GNv1,TPU,INT8,slow,1,10.03,5.721310,99.741000,174.785000,312.189330,547.077050,69.243400,88.445800,0.253000,0.253000,0.463000,3.130000,100.000000,l3,TPU_INT8_100%_GNv1,0.006099,INT8_TPU,TPU_INT8_GNv1_100.0
404,GNv1,TPU,INT8,fast,1,7.4,3.648520,135.087000,274.084000,422.822310,857.882920,69.243400,88.445800,0.253000,0.253000,0.538000,3.130000,100.000000,l3,TPU_INT8_100%_GNv1,0.003890,INT8_TPU,TPU_INT8_GNv1_100.0
437,GNv1,TX2,FP16,maxp,1,9.93,6.163370,99.824500,169.338000,312.450685,530.027940,66.928000,87.832000,1.800000,4.700000,8.070000,3.130000,100.000000,l3,TX2_FP16_100%_GNv1,0.006571,FP16_TX2,TX2_FP16_GNv1_100.0
438,GNv1,TX2,FP16,maxp,2,17.06,10.619700,108.360000,192.363000,339.166800,602.096190,66.928000,87.832000,1.800000,4.700000,8.280000,6.260000,100.000000,l3,TX2_FP16_100%_GNv1,0.011321,FP16_TX2,TX2_FP16_GNv1_100.0
